# Основные шаги для интеграции Fitbit API:


## Шаг 1: Регистрация приложения в Fitbit API
Зарегистрируйте своё приложение на Fitbit Developer Portal. Вам нужно будет указать:
- Имя приложения.
- URL-адрес перенаправления (для получения токена аутентификации).
- Описание приложения.
После регистрации вы получите **Client ID** и **Client Secret** для аутентификации через **OAuth 2.0**.

## Шаг 2: Аутентификация через OAuth 2.0  

- Fitbit API использует OAuth 2.0 для аутентификации. Пользователь должен предоставить разрешение на доступ к своим данным.
- После успешной аутентификации ваше приложение сможет запрашивать данные пользователя, такие как шаги, частота сердечных сокращений, калории и сон.

Пример кода для аутентификации через OAuth 2.0:

In [ ]:
import requests
from requests.auth import HTTPBasicAuth
import json

# Fitbit API credentials
client_id = 'your_client_id'
client_secret = 'your_client_secret'
redirect_uri = 'your_redirect_uri'  # Example: 'https://yourapp.com/callback'

# Step 1: Redirect user to authorize page
auth_url = f"https://www.fitbit.com/oauth2/authorize?response_type=code&client_id={client_id}&redirect_uri={redirect_uri}&scope=activity%20heartrate%20sleep"
print(f"Please visit this URL to authorize your app: {auth_url}")

# Step 2: User provides the authorization code
auth_code = input("Enter the authorization code here: ")

# Step 3: Exchange authorization code for access token
token_url = 'https://api.fitbit.com/oauth2/token'
response = requests.post(token_url,
                         data={
                             'client_id': client_id,
                             'grant_type': 'authorization_code',
                             'redirect_uri': redirect_uri,
                             'code': auth_code
                         },
                         auth=HTTPBasicAuth(client_id, client_secret))

tokens = response.json()
access_token = tokens['access_token']
print(f"Access token: {access_token}")


## Шаг 3: Получение данных из Fitbit API  

- Fitbit предоставляет различные эндпоинты для получения данных о физической активности, ЧСС, калориях и т.д.
- Можно использовать такие запросы, как получение данных о шагах за последние 7 дней, данных о ЧСС в реальном времени и т.д.

Примеры запросов для получения данных:

In [ ]:
# Запрос данных о шагах за последние 7 дней
headers = {
    'Authorization': f'Bearer {access_token}'
}
steps_url = 'https://api.fitbit.com/1/user/-/activities/steps/date/today/7d.json'
response = requests.get(steps_url, headers=headers)
steps_data = response.json()

# Пример вывода данных о шагах
for day in steps_data['activities-steps']:
    print(f"Date: {day['dateTime']}, Steps: {day['value']}")


In [ ]:
# Запрос данных о ЧСС
heartrate_url = 'https://api.fitbit.com/1/user/-/activities/heart/date/today/1d.json'
response = requests.get(heartrate_url, headers=headers)
heartrate_data = response.json()

# Пример вывода данных о ЧСС
for record in heartrate_data['activities-heart']:
    print(f"Date: {record['dateTime']}, Resting Heart Rate: {record['value']['restingHeartRate']}")


## Шаг 4: Интеграция данных в систему рекомендаций  

- Данные, полученные с помощью Fitbit API, будут поступать в вашу систему, где они могут быть использованы для обновления прогнозов и планов тренировок.
- Появится возможность анализировать данные в режиме реального времени и предоставлять динамические рекомендации на основе актуальных показателей пользователя.

Пример динамической интеграции:

In [ ]:
# Функция для динамического обновления тренировочного плана
def dynamic_training_plan(steps, heartrate, calories, sleep_minutes):
    if steps < 5000 and heartrate < 70:
        return "Сегодня рекомендуется увеличить активность: 30 минут кардио и 20 минут силовых упражнений."
    elif steps > 10000 and calories > 2000:
        return "Ваш уровень активности высок, продолжайте в том же духе! Рекомендуется добавить тренировки на выносливость."
    elif sleep_minutes < 360:
        return "Убедитесь, что у вас достаточно сна. Рекомендуется сделать акцент на восстановление и расслабляющие упражнения."
    else:
        return "Поддерживайте текущий уровень активности и следите за показателями ЧСС и сна."

# Пример вызова функции с данными от Fitbit API
steps_today = 8000  # Получено из Fitbit API
heartrate_today = 72  # Получено из Fitbit API
calories_today = 2200  # Получено из Fitbit API
sleep_today = 420  # Получено из Fitbit API

recommendation = dynamic_training_plan(steps_today, heartrate_today, calories_today, sleep_today)
print(recommendation)


# Шаг 5: Расширение рекомендаций на основе данных
Вы можете расширить возможности системы рекомендаций, добавив больше данных и усложнив логику:  
- Сравнение прогресса за несколько недель.
- Отслеживание долгосрочных тенденций в активности и восстановлении.
- Адаптация тренировочных планов в зависимости от времени года или событий (например, подготовка к марафону).

## Заключение:
- Интеграция Fitbit API позволит вашей системе получать данные в реальном времени, что существенно улучшит качество прогнозов и рекомендаций.
- Аутентификация через OAuth 2.0 обеспечивает безопасный доступ к данным пользователя.
- Запросы к API позволяют собирать данные о шагах, ЧСС, калориях и сне, которые могут использоваться для построения индивидуальных планов тренировок.
- Динамическая адаптация тренировок на основе данных пользователя позволяет системе стать более гибкой и персонализированной.